<a href="https://colab.research.google.com/github/21Lotus/Guitar-Chords-recognition/blob/master/SAD_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
import librosa
!pip install kapre
import kapre
import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd
from datetime import datetime
from kapre.time_frequency import STFT, Magnitude, MagnitudeToDecibel
from tensorflow.keras.utils import plot_model

In [ ]:
from scipy.io import wavfile
from collections import defaultdict, Counter
from scipy import signal
import random as rn
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import librosa
from kapre.augmentation import SpecAugment
from kapre.composed import get_melspectrogram_layer
import IPython.display as ipd
import matplotlib

In [ ]:
%%capture
!pip install kapre
import sklearn
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from kapre.composed import get_melspectrogram_layer, get_log_frequency_spectrogram_layer
from kapre.time_frequency import STFT, Magnitude, ApplyFilterbank, MagnitudeToDecibel
from sklearn.model_selection import train_test_split

In [ ]:
import librosa
import librosa.display
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import imgaug.augmenters as iaa

In [ ]:
%%capture
!pip -q install streamlit > /dev/null
!pip -q install pyngrok > /dev/null
!pip install streamlit-audiorec
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install PyAudio
!pip install pydub

import io
import pyaudio
import pydub
import numpy as np
from pydub import AudioSegment
from pydub.playback import play
from st_audiorec import st_audiorec
import tensorflow as tf
import pandas as pd
import numpy as np
from pyngrok import ngrok

In [ ]:
!ngrok authtoken _____

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


# Data Preprocessing

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/AzSEF24/masterDataSheet.csv')
df.head(64)

,Unnamed: 0,Passage 1,Passage 2,BDI-Score,Depressed/Not Depressed
0,0,/content/drive/MyDrive/Passage 1/1_P1.wav,/content/drive/MyDrive/Passage 2/1_P2.wav,18,1
1,1,/content/drive/MyDrive/Passage 1/2_P1.wav,/content/drive/MyDrive/Passage 2/2_P2.wav,21,1
2,2,/content/drive/MyDrive/Passage 1/3_P1.wav,/content/drive/MyDrive/Passage 2/3_P2.wav,0,0
3,3,/content/drive/MyDrive/Passage 1/4_P1.wav,/content/drive/MyDrive/Passage 2/4_P2.wav,11,0
4,4,/content/drive/MyDrive/Passage 1/5_P1.wav,/content/drive/MyDrive/Passage 2/5_P2.wav,6,0
...,...,...,...,...,...
59,59,/content/drive/MyDrive/Passage 1/60_P1.wav,/content/drive/MyDrive/Passage 2/60_P2.wav,23,1
60,60,/content/drive/MyDrive/Passage 1/61_P1.wav,/content/drive/MyDrive/Passage 2/61_P2.wav,20,1
61,61,/content/drive/MyDrive/Passage 1/62_P1.wav,/content/drive/MyDrive/Passage 2/62_P2.wav,27,1
62,62,/content/drive/MyDrive/Passage 1/63_P1.wav,/content/drive/MyDrive/Passage 2/63_P2.wav,20,1


In [ ]:
def extract_mfcc(filename, desired_length, n_mfcc):
    # Load audio file
    y, sr = librosa.load(filename)

    # Extract MFCC features
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)

    # Trim or pad the temporal dimension
    mfcc_processed = librosa.util.fix_length(mfcc, size=desired_length, axis=1)

    return mfcc_processed


#Data Pre-Processing
X = []
y = []
dual_representation = True

for i, row in df.iterrows():
  mfcc = extract_mfcc(row['Passage 1'], 1300, 13)
  mfcc2 = extract_mfcc(row['Passage 2'], 4000, 13)
  score = row['Depressed/Not Depressed']
  if dual_representation:
    features = np.concatenate((mfcc, mfcc2), axis=1)
    X.append(features)
    y.append(score)
  else:
    X.append(mfcc)
    # X.append(mfcc2)
    y.append(score)
    # y.append(score)

X = np.array(X)
y = np.array(y)
print(X.shape)
print(y.shape)

(64, 13, 5300)
(64,)


In [ ]:
X = X.transpose(0, 2, 1)
print(X.shape)

(64, 5300, 13)


In [ ]:
import imgaug.augmenters as iaa

def augment_audio_data(X, y, num_augmented_samples=5):
    augmented_samples = []
    scores = []

    augmentation_pipeline = iaa.Sequential([
        iaa.BlendAlphaFrequencyNoise(
            exponent=(-2, 2),
            foreground=iaa.Multiply((0.8, 1.2), per_channel=True),
            background=iaa.LinearContrast((0.8, 1.2))
        )
    ])

    for index, audio_sample in enumerate(X):
        label = y[index]
        for _ in range(num_augmented_samples):
            augmented_sample = augmentation_pipeline.augment_image(audio_sample)
            augmented_samples.append(augmented_sample)
            show_augmented_mfcc(augmented_samples, num_augmented_samples)
            scores.append(label)
    augmented_samples = np.array(augmented_samples)
    scores = np.array(scores)

    X_combined = np.concatenate([X, augmented_samples], axis=0)

    y_combined = np.concatenate([y, scores], axis=0)

    return X_combined, y_combined

In [ ]:
def show_augmented_mfcc(augmented_mfccs, num_augmented_samples, figsize=(30, 30)):
    # Display the augmented mel spectrograms
    plt.figure(figsize=figsize)  # Set the overall figure size

    for i, augmented_mfcc in enumerate(augmented_mfccs):
        plt.subplot(2, num_augmented_samples + 1, i + 2)
        librosa.display.specshow(
            y_axis='coefficients',
            x_axis='time',
            sr=22050,  # replace your_sampling_rate with the actual sampling rate
            hop_length=256  # replace your_hop_length with the actual hop length
        )
        plt.colorbar()
        plt.title(f'Augmented #{i + 1}')

    # Adjust layout
    plt.show()

# CNN Model

In [ ]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
import imgaug.augmenters as iaa

import tensorflow as tf
import kapre
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2)

X_combined, y_combined = augment_audio_data(X_train, y_train, 2)

kwargs = {
    'sample_rate': 22050,
    'n_freq': 512 // 2 + 1,
    'n_mels': 13,
    'f_min': 0.0,
    'f_max': 8000
  }

input_shape = (5300, 13)

model = Sequential()
model.add(InputLayer(input_shape=input_shape))

# STFT -> Magnitude
model.add(STFT(n_fft=512, win_length=512, hop_length=256, input_data_format='channels_last',
               output_data_format='channels_last'))
model.add(Magnitude())

# ApplyFilterbank (Mel-filterbank)
model.add(ApplyFilterbank(type='mel', filterbank_kwargs=kwargs))

# MagnitudeToDecibel
model.add(MagnitudeToDecibel())

# Batch Normalization
model.add(BatchNormalization())

# Conv2D and Max Pooling Layers (with adjustments)
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', use_bias=True))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same', use_bias=True))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(8, kernel_size=(3, 3), activation='relu', padding='same', use_bias=True, kernel_regularizer = tf.keras.regularizers.l2(l=0.0001)))

#Flatten, dropout, and dense layers
model.add(Flatten())
model.add(Dropout(0.6))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

opt = keras.optimizers.Adam(learning_rate=1e-4)

# Let's train the model using adam
model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])


checkpoint = ModelCheckpoint('best_model.h5', monitor="val_accuracy", verbose=1, save_best_only=True, mode="max")

# Train the CNN and plot accuracy.
history = model.fit(X_combined, y_combined, epochs=100, batch_size=6, validation_split = 0.2, shuffle=True, callbacks=[checkpoint])

import matplotlib.pyplot as plt
def show_final_history(history):
    fig, ax = plt.subplots(1, 2, figsize=(15,5))
    ax[0].set_title('loss')
    ax[0].plot(history.epoch, history.history["loss"], label="Train loss")
    ax[0].plot(history.epoch, history.history["val_loss"], label="Validation loss")
    ax[1].set_title('acc')
    ax[1].plot(history.epoch, history.history["accuracy"], label="Train acc")
    ax[1].plot(history.epoch, history.history["val_accuracy"], label="Validation acc")
    ax[0].legend()
    ax[1].legend()

show_final_history(history)

Accuracy Check

In [ ]:
from keras.models import load_model
from keras.utils import custom_object_scope
import imgaug.augmenters as iaa

model = load_model('best_model.h5', custom_objects={'STFT':kapre.time_frequency.STFT,
                       'Magnitude':kapre.time_frequency.Magnitude,
                       'ApplyFilterbank': kapre.time_frequency.ApplyFilterbank,
                       'MagnitudeToDecibel':kapre.time_frequency.MagnitudeToDecibel})
#tf.keras.utils.get_custom_objects().update({'ImgAugAugmentation': ImgAugAugmentation})

predictions = model.evaluate(X_test, y_test)

1/1 [==============================] - 0s 408ms/step - loss: 0.4659 - accuracy: 0.9231
